In [1]:
from deltalake import DeltaTable, write_deltalake

## Pandas

In [2]:
import pandas as pd

In [14]:
woods = [
    {'name': 'oak', 'type': 'hard'},
    {'name': 'cedar', 'type': 'soft'}
]
woods_df = pd.DataFrame(woods)
write_deltalake("delta_lake/woods", woods_df, mode='overwrite')

In [10]:
pd_df = DeltaTable("delta_lake/woods").to_pandas()
pd_df

,name,type
0,oak,hard
1,cedar,soft


## Polars

In [ ]:
import polars as pl

In [12]:
df = pl.DataFrame([{"name": 'polar bear', 'environment':'cold'},
                  {"name": 'brown bear', 'environment':'many'},
                  {"name": 'panda bear', 'environment':'jungle'}])
df.write_delta("delta_lake/bears", mode='overwrite')

In [13]:
pl_df = pl.read_delta("delta_lake/bears")
pl_df

name,environment
str,str
"""polar bear""","""cold"""
"""brown bear""","""many"""
"""panda bear""","""jungle"""


## DuckDB

In [29]:
import duckdb

bears_dataset = DeltaTable("delta_lake/bears").to_pyarrow_dataset()
woods_dataset = DeltaTable("delta_lake/woods").to_pyarrow_dataset()

In [30]:

quack = duckdb.arrow(bears_dataset)
test = quack.query('bears_dataset', "SELECT * FROM bears_dataset WHERE name = 'polar bear'").fetchall()

In [33]:
# Create a DuckDB connection
con = duckdb.connect()

# Register the Arrow dataset with DuckDB
con.execute("""SELECT *
                FROM
                bears_dataset
                FULL OUTER JOIN woods_dataset USING ('name')
            ;""").fetch_arrow_table()

,name,environment,type
0,oak,None,hard
1,cedar,None,soft
2,polar bear,cold,None
3,panda bear,jungle,None
4,brown bear,many,None
